In [4]:
from eolearn.core import EOPatch, FeatureType, SaveToDisk, LoadFromDisk
from eolearn.io import ImportFromTiff

import numpy as np
import matplotlib.pyplot as plt
import rasterio

import glob

from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
load_task = LoadFromDisk(folder='')
save_task = SaveToDisk(folder='/home/mlubej/fun/miha_zemlja/data/eopatches/')

eopatch_names = glob.glob('/home/mlubej/fun/miha_zemlja/data/eopatches/*')

In [3]:
eopatch = load_task.execute(eopatch_folder=eopatch_names[0])

In [12]:
np.random.seed(5)


dataset = eopatch.data_timeless['BANDS']
old_shape = dataset.shape
dataset = dataset.reshape(old_shape[0]*old_shape[1],old_shape[2])

_, subset = train_test_split(dataset, test_size = 0.0001)


In [13]:
len(subset)

36781

In [19]:
estimators = [('k_means_8', KMeans(n_clusters=8)),
              ('k_means_3', KMeans(n_clusters=3))]

est = estimators[1][1]

In [20]:
res = est.fit(subset)

In [21]:
res.cluster_centers_

array([[122.64313201, 161.1984273 ,  84.53463276],
       [196.39582427, 202.40278382, 170.66420183],
       [ 79.03953092, 120.79146964,  53.49905428]])

In [22]:
np.unique(res.labels_)

array([0, 1, 2], dtype=int32)

In [24]:
res.predict(dataset)

MemoryError: 

In [ ]:


fignum = 1
titles = ['8 clusters', '3 clusters', '3 clusters, bad initialization']
for name, est in estimators:
    fig = plt.figure(fignum, figsize=(4, 3))
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    est.fit(X)
    labels = est.labels_

    ax.scatter(X[:, 3], X[:, 0], X[:, 2],
               c=labels.astype(np.float), edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('Petal width')
    ax.set_ylabel('Sepal length')
    ax.set_zlabel('Petal length')
    ax.set_title(titles[fignum - 1])
    ax.dist = 12
    fignum = fignum + 1

# Plot the ground truth
fig = plt.figure(fignum, figsize=(4, 3))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

for name, label in [('Setosa', 0),
                    ('Versicolour', 1),
                    ('Virginica', 2)]:
    ax.text3D(X[y == label, 3].mean(),
              X[y == label, 0].mean(),
              X[y == label, 2].mean() + 2, name,
              horizontalalignment='center',
              bbox=dict(alpha=.2, edgecolor='w', facecolor='w'))
# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 2, 0]).astype(np.float)
ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=y, edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('Petal width')
ax.set_ylabel('Sepal length')
ax.set_zlabel('Petal length')
ax.set_title('Ground Truth')
ax.dist = 12

fig.show()